# **[ 数値データの処理 / preprocess numerical data ]**

　冒頭でも述べましたが、データ処理はエラー値の修正だけでなく、データ全体を把握する役割も兼ねています。処理プロセス毎に生じるデータ差分を把握しながら、後のパターン分析や特徴量生成に繋がるフォーマットへとデータを正しく変換することが重要であると考えています。
 
　今回は、タイプの異なる4つの金融商品（株、FX、仮想通貨、金利）の数値データを扱うため、エラー値の定義には注意が必要です。例えば、株価は0より小さくなりませんが、金利はマイナスの値を取ることができます。FXの小数桁数は3〜5桁が一般的ですが、仮想通貨は更に細かい小数桁数を取ることがあります。それぞれの商品特性を踏まえた上で、データを正しく変換する必要があります。

# **データの準備**

　マーケットデータ取得には、データプロバイダーのAPIを使用します。尚、プロバイダー毎のデータ相違、及び有料データの優位点については、このnotebookでは特段触れません。

- [Alpha Vantage](https://www.alphavantage.co)
- [Tiingo](https://www.tiingo.com)
- [twelvedata](https://twelvedata.com)

　上記プロバイダーは執筆時点で無料APIを提供していますが、以下のcodeから実際にデータを取得するには各プロバイダーのAPIキーが別途必要となります。詳細は、リンクからDocsを参照してください。

---
 
 - [yfinance](https://github.com/ranaroussi/yfinance)
 
　株式関連であれば、オープンソースツールのyfinanceを使ったデータ取得も可能です。APIキーを別途取得する必要はありません。

### **`stocks`**

In [15]:
import requests
import csv
import json
import pandas as pd
import yfinance as yf

# Alpha Vantage -> daily and intraday
def alpha_stocks(ticker=None):
    url = "https://www.alphavantage.co/query"
    params = {
            "function":"TIME_SERIES_INTRADAY_EXTENDED",
            "symbol":ticker,
            "interval":"60min", #1min', '5min', '15min', '30min', '60min'
            "slice":"year2month12",
            "apikey":"Enter Your API Key"
            }
    res = requests.get(url, params=params)
    decoded = res.content.decode("utf-8")
    data = list(csv.reader(decoded.splitlines(), delimiter=','))
    df = pd.DataFrame(data[1:], columns=data[0])
    return df


# Tiingo -> daily data
def tiingo_stocks(ticker=None):
    headers = {
        "Content-Type": "application/json",
        "Authorization" : "Enter Your API Key"
        }
    url = "https://api.tiingo.com/tiingo/daily/"
    startdate = "2022-01-01"
    enddate = "2022-11-28"
    resample = "daily"
    sort = "-date"
    format_ = "csv"

    params = f"{ticker}/prices?startDate={startdate}&endDate={enddate}&format={format_}&sort={sort}&resampleFreq={resample}"
    res = requests.get(url + params, headers=headers)
    decode_content = res.content.decode('utf-8')
    data = list(csv.reader(decode_content.splitlines(), delimiter=','))
    df = pd.DataFrame(data[1:], columns=data[0])
    return df


# twelvedata
def twelve_stocks(ticker=None):
    url = "https://api.twelvedata.com/time_series?"
    params = {
        "symbol":ticker,
        "country":"US",
        "type":"Stock", #Index, ETF, REIT
        "interval":"1h", #1min, 5min, 15min, 30min, 45min, 1h, 2h, 4h, 1day, 1week, 1month
        "outputsize":5000,
        "format":"csv", #json,
        "apikey":"Enter Your API Key"
    }
    res = requests.get(url, params)
    decode_content = res.content.decode('utf-8')
    data = list(csv.reader(decode_content.splitlines(), delimiter=';'))
    df = pd.DataFrame(data[1:], columns=data[0])
    return df


# yfinance
def yfinance_stocks(ticker=None):
    data = yf.Ticker(ticker)
    data_hist = data.history(interval="60m", start="2022-01-01", end="2022-11-28")
    return data_hist

In [2]:
alpha_stocks("AAPL")
# tiingo_stocks("AAPL")
# twelve_stocks("AAPL")
# yfinance_stocks("AAPL")

,{
0,"""Error Message"": ""the parameter apikey is ..."
1,}


### **`FX`**

In [16]:
import requests
import csv
import json
import pandas as pd

# Alpha Vantage
def alpha_fx(ticker=None):
    url = "https://www.alphavantage.co/query"
    params = {
            "function":"FX_DAILY",
            "from_symbol":ticker.split("/")[0],
            "to_symbol":ticker.split("/")[1],
            "outputsize":"compact",
            "datatype":"csv",
            "apikey":"Enter Your API Key"
            }
    res = requests.get(url, params=params)
    decoded = res.content.decode("utf-8")
    data = list(csv.reader(decoded.splitlines(), delimiter=','))
    df = pd.DataFrame(data[1:], columns=data[0])
    return df


# Tiingo
def tiingo_fx(ticker=None):
    headers = {
        "Content-Type": "application/json",
        "Authorization" : "Enter Your API Key"
        }
    url = "https://api.tiingo.com/tiingo/fx/"
    resample = "60min" #min, hour, and day
    format_ = "csv"
    params = f"{ticker}/prices?resampleFreq={resample}&format={format_}"
    res = requests.get(url + params, headers=headers)
    decode_content = res.content.decode('utf-8')
    data = list(csv.reader(decode_content.splitlines(), delimiter=','))
    df = pd.DataFrame(data[1:], columns=data[0])
    return df


# twelvedata
def twelve_fx(ticker=None):
    url = "https://api.twelvedata.com/time_series?"
    params = {
        "symbol":ticker,
        "interval":"1h",
        "outputsize":5000,
        "format":"csv",
        "apikey":"Enter Your API Key"
    }
    res = requests.get(url, params)
    decode_content = res.content.decode('utf-8')
    data = list(csv.reader(decode_content.splitlines(), delimiter=';'))
    df = pd.DataFrame(data[1:], columns=data[0])
    return df

In [18]:
# alpha_fx("USD/JPY")
# tiingo_fx("USDJPY")
# twelve_fx("USD/JPY")

,datetime,open,high,low,close
0,2022-11-30 22:00:00,138.84500,138.94501,138.78500,138.89000
1,2022-11-30 21:00:00,138.76500,139.04500,138.73500,138.85500
2,2022-11-30 20:00:00,138.62500,138.87000,138.60001,138.77499
3,2022-11-30 19:00:00,138.53500,138.85500,138.45500,138.62500
4,2022-11-30 18:00:00,138.59500,138.60500,138.37500,138.53500
...,...,...,...,...,...
4995,2022-02-11 23:00:00,116.00500,116.02000,115.89500,115.94500
4996,2022-02-11 22:00:00,116.01000,116.03500,115.97000,116.00000
4997,2022-02-11 21:00:00,116.05500,116.07000,115.98500,116.02500
4998,2022-02-11 20:00:00,116.05000,116.08500,116.02000,116.05000


### **`Cryptos`**

In [79]:
import requests
import csv
import json
import pandas as pd

# Alpha Vantage
def alpha_cryptos(ticker=None):
    url = "https://www.alphavantage.co/query"
    params = {
            "function":"DIGITAL_CURRENCY_DAILY",
            "symbol":ticker,
            "market":"USD",
            "datatype":"csv",
            "apikey":"Enter Your API Key"
            }
    res = requests.get(url, params=params)
    decoded = res.content.decode("utf-8")
    data = list(csv.reader(decoded.splitlines(), delimiter=','))
    df = pd.DataFrame(data[1:], columns=data[0])
    return df


# Tiingo
def tiingo_cryptos(ticker=None):
    headers = {
        "Content-Type": "application/json",
        "Authorization" : "Enter Your API Key"
        }
    url = "https://api.tiingo.com/tiingo/crypto/"
    resample = "1hour"
    params = f"prices?tickers={ticker}&resampleFreq={resample}"
    res = pd.DataFrame(requests.get(url + params, headers=headers).json())
    df = pd.DataFrame(res["priceData"][0])
    return df


# twelvedata
def twelve_cryptos(ticker=None):
    url = "https://api.twelvedata.com/time_series?"
    params = {
        "symbol":ticker,
        "interval":"1h",
        "outputsize":5000,
        "format":"csv",
        "apikey":"Enter Your API Key"
    }
    res = requests.get(url, params)
    decode_content = res.content.decode('utf-8')
    data = list(csv.reader(decode_content.splitlines(), delimiter=';'))
    df = pd.DataFrame(data[1:], columns=data[0])
    return df

In [80]:
# alpha_cryptos("BTC")
# tiingo_cryptos("BTCUSD")
# twelve_cryptos("BTC/USD")

,datetime,open,high,low,close
0,2022-11-30 12:00:00,16878.91992,16887.42969,16870.80078,16870.80078
1,2022-11-30 11:00:00,16868.83008,16899.72070,16868.83008,16879.61914
2,2022-11-30 10:00:00,16879.42969,16897.24023,16870.07031,16874.77930
3,2022-11-30 09:00:00,16890.14062,16899.50977,16847.02930,16871.66016
4,2022-11-30 08:00:00,16881.48047,16927.01953,16877.15039,16889.82031
...,...,...,...,...,...
4995,2022-05-05 10:00:00,39478.87891,39597.71875,39442.14844,39490.71094
4996,2022-05-05 09:00:00,39444.35156,39549.07812,39410.23047,39477.46094
4997,2022-05-05 08:00:00,39577.80078,39619.76953,39433.57031,39462.55859
4998,2022-05-05 07:00:00,39517.55078,39604.32031,39503.08984,39582.76172


# **処理プロセス**

　実装するデータ処理プロセスは、以下の項目になります。

1. ### **`日時チェック / date`**
- datetimeデータを抽出
2. ### **`null（1回目）/ null(first)`**
- 値カラムが全てnanのrowを削除
3. ### **`マイナス値 / negative value`**
- 値カラムにマイナスが含まれるrowを削除
4. ### **`ゼロ / zero value`**
- 値カラムに０が含まれるrowを削除
5. ### **`小数桁数 / dicimal`**
- 値カラムの小数桁数を条件毎にround
6. ### **`値 / value`**
- 高値カラムに高値以外の値があるrowを削除
- 安値カラムに安値以外の値があるrowを削除
- 終値 - 始値が閾値を超えるrow数をカウント
7. ### **`null（2回目）/ null(second)`**
- 残るnullを検証し、修正or削除

### **`1. 日時チェック / date check`**
　これは全てのデータカテゴリーに共通する実装となります。同じdate rangeでも、API毎に取得できるデータrow数は異なることが多いため、

### **`2. null（1回目）/ null(first)`**

In [ ]:
　この項目も全てのデータカテゴリーに共通する実装となります。

### **`3. マイナス値 / negative value`**

### **`4. ゼロ / zero value`**

### **`5. 小数桁数 / dicimal`**

### **`6. 値 / value`**

### **`7. null（2回目）/ null(second)`**

### **`処理データの差分 / data difference`**